In [104]:
import pandas as pd
import os
import numpy as np

filenames = os.listdir('./team_stats/')
filenames_stats = filenames[1::2]
filenames_logs = filenames[0::2]
filenames_stats[19], filenames_logs[19]

('lions_stats.csv', 'lions_logs.csv')

In [116]:
df_log_lions = pd.read_csv(f'./team_stats/{filenames_logs[19]}')
df_stats_lions = pd.read_csv(f'./team_stats/{filenames_stats[19]}')
df_log_lions.columns
df_stats_lions.columns

Index(['Week', 'Day', 'Date', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'OT',
       'Rec', 'Unnamed: 8', 'Opp', 'Tm', 'Opp.1', '1stD', 'TotYd', 'PassY',
       'RushY', 'TO', '1stD.1', 'TotYd.1', 'PassY.1', 'RushY.1', 'TO.1',
       'Offense', 'Defense', 'Sp. Tms'],
      dtype='object')

In [118]:
df_log = pd.read_csv(f'./team_stats/{filenames_logs[20]}')
df_stats = pd.read_csv(f'./team_stats/{filenames_stats[20]}')

logs_equal = df_log.columns == df_log_lions.columns
stats_equal = df_stats.columns == df_stats_lions.columns
stats_equal

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [81]:
drop_columns_stats = ['Day', 'Date', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8', 'OT', 'PassY', 'RushY',
        'Opp', 'Tm', 'Opp.1', 'TotYd', ]


In [82]:
drop_columns_logs = [
    'Unnamed: 3', 'Unnamed: 4', 'OT', 'XPM', 'XPA', 'Pnt','Yds.3'
]

In [83]:
rename_dict_logs = {
    'Week':'week',
    'Day':'day',
    'Date':'date',
    'Unnamed: 6':'home/away',
    'Opp':'opponent',
    'Tm':'score_team',
    'Opp.1':'score_opp',
    'Cmp':'pass_cmp_off',
    'Att':'pass_att_off',
    'Yds':'pass_yds_off',
    'TD':'pass_tds_off',
    'Int':'ints_off',
    'Sk':'sacks_off',
    'Yds.1':'sacks_yds_off',
    'Y/A':'pass_yds_per_att',
    'NY/A':'pass_net_yds_per_att',
    'Cmp%':'pass_cmp_prcnt',
    'Rate':'pass_qb_rate',
    'Att.1':'rush_att_off',
    'Yds.2':'rush_yds_off',
    'Y/A.1':'rush_yds_per_att',
    'TD.1':'rush_tds_off',
    'FGM':'fg_made',
    'FGA':'fg_att',
    '3DConv':'3d_made',
    '3DAtt':'3d_att',
    '4DConv':'4d_made',
    '4DAtt':'4d_att',
    'ToP': 'time_poss'
}

In [84]:
df_log_1 = df_log.drop(drop_columns_logs, axis=1)
df_log_1 = df_log_1.rename(columns = rename_dict_logs)
df_log_1.columns

Index(['week', 'day', 'date', 'home/away', 'opponent', 'score_team',
       'score_opp', 'pass_cmp_off', 'pass_att_off', 'pass_yds_off',
       'pass_tds_off', 'ints_off', 'sacks_off', 'sacks_yds_off',
       'pass_yds_per_att', 'pass_net_yds_per_att', 'pass_cmp_prcnt',
       'pass_qb_rate', 'rush_att_off', 'rush_yds_off', 'rush_yds_per_att',
       'rush_tds_off', 'fg_made', 'fg_att', '3d_made', '3d_att', '4d_made',
       '4d_att', 'time_poss'],
      dtype='object')

In [85]:
rename_dict_stats = {
    'Week':'wk',
    'Rec':'record',
    '1stD':'1st_down_off',
    'TO':'turnover_off',
    '1stD.1':'1st_down_def',
    'TotYd.1':'tot_yds_def',
    'PassY.1':'pass_yds_def',
    'RushY.1':'rush_yds_def',
    'TO.1':'turnover_def',
    'Offense':'exp_off',
    'Defense':'exp_def',
    'Sp. Tms':'exp_st',
}

In [98]:
def get_trimmed_df(filename, drop_columns, rename_dict):
    # team = filename.split('_')[0]
    # print(team)
    df = pd.read_csv(f'./team_stats/{filename}')
    df = df.drop(drop_columns, axis=1)
    df = df.rename(columns=rename_dict)
    return df

In [99]:
def combine_trimmed_stats_w_logs(df_stats, df_logs):
    # if team_stats != team_logs:
    #     print('TEAMS DONT MATCH')
    #     return
    combo_df = pd.concat([df_logs, df_stats], axis=1,)
    return combo_df


In [100]:

def get_home_and_away_columns(df):
    df['home/away'] = np.where(
        df['home/away'] == '@', 
        'away', 
        'home'
    )
    
    return df

def get_short_team_name(full_team_name):
    # print(full_team_name)
    short_name = full_team_name.rsplit(None, 1)[-1]
    lc_short_name = short_name.lower()
    return lc_short_name

In [121]:
def clean_and_concat_stats_logs(team):
    stats_filename = f'{team}_stats.csv'
    logs_filename = f'{team}_logs.csv'
    
    # get stats df
    df_stats = get_trimmed_df(stats_filename, drop_columns_stats, rename_dict_stats)
    
    # get logs df
    df_logs = get_trimmed_df(logs_filename, drop_columns_logs, rename_dict_logs)
    
    # combine dfs using concat
    df_combo = combine_trimmed_stats_w_logs(df_stats, df_logs)
    
    # set home/away columns replacing NaN and @ with home and away
    df_combo = get_home_and_away_columns(df_combo)
    
    # set opponent column to team name only, remove city
    df_combo['opponent'] = df_combo['opponent'].apply(lambda x: get_short_team_name(x))
    
    # check that weeks in both dataframes match after join
    wks_equal = df_combo['week'].equals(df_combo['wk'])
    if wks_equal != True:
        print('ERROR: weeks did not match in join')
        return 
    # weeks match so drop one of the weeks columns
    df_combo.drop(['wk'], axis=1)
    
    # create team column
    df_combo['team'] = team
    
    # save df to csv file
    df_combo.to_csv(f'./team_stats_combo/{team}_combo_stats.csv')
    
    return df_combo

In [102]:
df_lions = clean_and_concat_stats_logs('lions')

KeyError: "['Day', 'Date', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'OT', 'PassY', 'RushY', 'Opp', 'Tm', 'Opp.1', 'TotYd'] not found in axis"

In [92]:
# df_texans.columns

Index(['week', 'day', 'date', 'home/away', 'opponent', 'score_team',
       'score_opp', 'pass_cmp_off', 'pass_att_off', 'pass_yds_off',
       'pass_tds_off', 'ints_off', 'sacks_off', 'sacks_yds_off',
       'pass_yds_per_att', 'pass_net_yds_per_att', 'pass_cmp_prcnt',
       'pass_qb_rate', 'rush_att_off', 'rush_yds_off', 'rush_yds_per_att',
       'rush_tds_off', 'fg_made', 'fg_att', '3d_made', '3d_att', '4d_made',
       '4d_att', 'time_poss', 'wk', 'record', '1st_down_off', 'turnover_off',
       '1st_down_def', 'tot_yds_def', 'pass_yds_def', 'rush_yds_def',
       'turnover_def', 'exp_off', 'exp_def', 'exp_st', 'team'],
      dtype='object')

In [61]:
from team_names import team_names
print(team_names)

['patriots', 'cowboys', 'giants', 'bills', 'lions', 'texans', 'ravens', 'lions', 'vikings', 'eagles', 'falcons', 'bears', '49ers', 'rams', 'bengals', 'raiders', 'buccaneers', 'browns', 'broncos', 'jaguars', 'jets', 'commanders', 'titans', 'steelers', 'packers', 'dolphins', 'seahawks', 'chiefs', 'chargers', 'colts', 'saints', 'panthers', 'cardinals']


In [84]:
for team in team_names[8:]:
    print(team)
    clean_and_concat_stats_logs(team)

vikings
eagles
falcons
bears
49ers
rams
bengals
raiders
buccaneers
browns
broncos
jaguars
jets
commanders
titans
steelers
packers
dolphins
seahawks
chiefs
chargers
colts
saints
panthers
cardinals


In [122]:
clean_and_concat_stats_logs('lions')

,week,day,date,home/away,opponent,score_team,score_opp,pass_cmp_off,pass_att_off,pass_yds_off,...,turnover_off,1st_down_def,tot_yds_def,pass_yds_def,rush_yds_def,turnover_def,exp_off,exp_def,exp_st,team
0,1,Sun,September 11,home,eagles,35,38,21,37,205,...,1.0,27,455,239,216,NaN,15.67,-21.15,0.78,lions
1,2,Sun,September 18,home,commanders,36,27,20,34,234,...,NaN,21,396,308,88,1.0,7.13,-5.79,10.29,lions
2,3,Sun,September 25,away,vikings,24,28,25,41,277,...,1.0,30,373,250,123,1.0,8.25,-11.58,-1.93,lions
3,4,Sun,October 2,home,seahawks,45,48,27,40,375,...,2.0,27,555,320,235,1.0,22.12,-32.93,3.99,lions
4,5,Sun,October 9,away,patriots,0,29,19,35,211,...,2.0,22,364,188,176,1.0,-22.04,-10.08,3.38,lions
5,7,Sun,October 23,away,cowboys,6,24,21,26,195,...,5.0,21,330,191,139,1.0,-12.68,-4.39,-1.61,lions
6,8,Sun,October 30,home,dolphins,27,31,27,37,311,...,NaN,27,476,369,107,1.0,19.14,-25.47,0.59,lions
7,9,Sun,November 6,home,packers,15,9,14,26,137,...,1.0,19,389,283,106,3.0,-1.35,5.25,2.44,lions
8,10,Sun,November 13,away,bears,31,30,19,26,228,...,NaN,19,408,150,258,1.0,13.55,-9.38,2.09,lions
9,11,Sun,November 20,away,giants,31,18,17,26,165,...,NaN,25,413,324,89,3.0,9.36,-0.64,7.69,lions
